In [ ]:
!pip3 uninstall --yes torch torchaudio torchvision torchtext torchdata
!pip3 install torch torchaudio torchvision torchtext torchdata

Found existing installation: torch 2.3.1
Uninstalling torch-2.3.1:
  Successfully uninstalled torch-2.3.1
Found existing installation: torchaudio 2.3.1
Uninstalling torchaudio-2.3.1:
  Successfully uninstalled torchaudio-2.3.1
Found existing installation: torchvision 0.18.1
Uninstalling torchvision-0.18.1:
  Successfully uninstalled torchvision-0.18.1
Found existing installation: torchtext 0.18.0
Uninstalling torchtext-0.18.0:
  Successfully uninstalled torchtext-0.18.0
Found existing installation: torchdata 0.7.1
Uninstalling torchdata-0.7.1:
  Successfully uninstalled torchdata-0.7.1
  Using cached torch-2.3.1-cp310-cp310-manylinux1_x86_64.whl (779.1 MB)
  Using cached torchaudio-2.3.1-cp310-cp310-manylinux1_x86_64.whl (3.3 MB)
  Using cached torchvision-0.18.1-cp310-cp310-manylinux1_x86_64.whl (7.0 MB)
  Using cached torchtext-0.18.0-cp310-cp310-manylinux1_x86_64.whl (2.0 MB)
  Using cached torchdata-0.7.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (4.7 MB)


In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [ ]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

In [ ]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [ ]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [ ]:
# Import necessary libraries
import torch
import torch.nn as nn

# Get CPU, GPU, or MPS device for training
device = (
    "cuda"                                    # Check if CUDA (GPU) is available
    if torch.cuda.is_available()               # If CUDA is available, use it
    else "mps"                                 # Check if CUDA Managed MPS is available
    if torch.backends.mps.is_available()       # If MPS is available, use it
    else "cpu"                                 # Otherwise, use CPU
)
print(f"Using {device} device")                # Print the device being used

# Define the neural network model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()                     # Initialize the base class (nn.Module)

        # Flatten layer to convert 2D image data (28x28) to 1D tensor (784)
        self.flatten = nn.Flatten()

        # Sequential stack of linear and ReLU layers
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),             # 1st fully connected layer: 784 input features, 512 output features
            nn.ReLU(),                         # ReLU activation function
            nn.Linear(512, 512),               # 2nd fully connected layer: 512 input features, 512 output features
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),                         # ReLU activation function
            nn.Linear(256, 10)                 # 3rd fully connected layer: 512 input features, 10 output features (classes 0-9)
        )

    def forward(self, x):
        x = self.flatten(x)                    # Flatten the input tensor
        logits = self.linear_relu_stack(x)      # Pass the flattened input through the sequential layers
        return logits                          # Return the raw output (logits)

# Create an instance of the NeuralNetwork model and move it to the selected device
model = NeuralNetwork().to(device)

# Print the model structure
print(model)


Using cuda device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=256, bias=True)
    (5): ReLU()
    (6): Linear(in_features=256, out_features=10, bias=True)
  )
)


In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [ ]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)  # Total number of samples in the dataset
    model.train()                   # Set the model to training mode
    for batch, (X, y) in enumerate(dataloader):  # Iterate over batches in the dataloader
        X, y = X.to(device), y.to(device)       # Move data batches to the selected device (CPU or GPU)

        # Forward pass: compute predicted outputs by passing inputs to the model
        pred = model(X)

        # Compute the loss
        loss = loss_fn(pred, y)

        # Backpropagation: calculate gradients of the loss with respect to model parameters
        loss.backward()

        # Update the model parameters using the gradients and the chosen optimizer
        optimizer.step()

        # Zero the gradients to prevent them from accumulating in subsequent iterations
        optimizer.zero_grad()

        # Print training progress
        if batch % 100 == 0:
            # Retrieve the loss value and calculate the current batch size
            loss, current = loss.item(), (batch + 1) * len(X)
            # Print current batch loss and the number of processed samples out of the total
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


In [ ]:
def test(dataloader, model, loss_fn):
    # Get the total number of samples in the dataloader dataset
    size = len(dataloader.dataset)
    # Get the number of batches in the dataloader
    num_batches = len(dataloader)
    # Set the model to evaluation mode (disables dropout and batch normalization)
    model.eval()
    # Initialize variables to track the total loss and number of correct predictions
    test_loss, correct = 0, 0

    # Disable gradient calculation for efficiency during inference
    with torch.no_grad():
        # Iterate over batches in the dataloader
        for X, y in dataloader:
            # Move data and labels to the specified device (e.g., GPU or CPU)
            X, y = X.to(device), y.to(device)
            # Perform the forward pass to get model predictions
            pred = model(X)
            # Calculate the loss for the current batch and add it to the total loss
            test_loss += loss_fn(pred, y).item()
            # Count the number of correct predictions by comparing the predicted labels
            # (obtained via argmax) to the true labels and summing the results
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    # Calculate the average loss over all batches
    test_loss /= num_batches
    # Calculate the accuracy as the ratio of correct predictions to the total number of samples
    correct /= size

    # Print the test error, including accuracy (in percentage) and average loss
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")


In [ ]:
# Number of epochs to train the model
epochs = 100

# Loop over each epoch
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")

    # Train the model using the training dataloader
    train(train_dataloader, model, loss_fn, optimizer)

    # Test the model using the testing dataloader
    test(test_dataloader, model, loss_fn)

# Print a message when training and testing are done
print("Done!")


Epoch 1
-------------------------------
loss: 2.301228  [   64/60000]
loss: 2.299524  [ 6464/60000]
loss: 2.296021  [12864/60000]
loss: 2.297302  [19264/60000]
loss: 2.291381  [25664/60000]
loss: 2.286667  [32064/60000]
loss: 2.285662  [38464/60000]
loss: 2.280249  [44864/60000]
loss: 2.283090  [51264/60000]
loss: 2.273833  [57664/60000]
Test Error: 
 Accuracy: 24.5%, Avg loss: 2.278732 

Epoch 2
-------------------------------
loss: 2.279610  [   64/60000]
loss: 2.277636  [ 6464/60000]
loss: 2.270198  [12864/60000]
loss: 2.275278  [19264/60000]
loss: 2.265346  [25664/60000]
loss: 2.258263  [32064/60000]
loss: 2.262432  [38464/60000]
loss: 2.251502  [44864/60000]
loss: 2.254095  [51264/60000]
loss: 2.241081  [57664/60000]
Test Error: 
 Accuracy: 54.1%, Avg loss: 2.244518 

Epoch 3
-------------------------------
loss: 2.248223  [   64/60000]
loss: 2.245344  [ 6464/60000]
loss: 2.229025  [12864/60000]
loss: 2.238471  [19264/60000]
loss: 2.222417  [25664/60000]
loss: 2.206857  [32064/600